In [87]:
import os
import pandas as pd
import numpy as np

In [2]:
os.listdir()

['.ipynb_checkpoints', 'Loan Default EDA.ipynb']

In [88]:
data_path = 'C:/Users/H244746/Desktop/Deployment Exercise/archive/train.csv'
data = pd.read_csv(data_path)
print('Shape of the data is ', data.shape)
data.head(2)

Shape of the data is  (233154, 41)


,UNIQUEID,DISBURSED_AMOUNT,ASSET_COST,LTV,BRANCH_ID,SUPPLIER_ID,MANUFACTURER_ID,CURRENT_PINCODE_ID,DATE_OF_BIRTH,EMPLOYMENT_TYPE,...,SEC_SANCTIONED_AMOUNT,SEC_DISBURSED_AMOUNT,PRIMARY_INSTAL_AMT,SEC_INSTAL_AMT,NEW_ACCTS_IN_LAST_SIX_MONTHS,DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS,AVERAGE_ACCT_AGE,CREDIT_HISTORY_LENGTH,NO_OF_INQUIRIES,LOAN_DEFAULT
0,420825,50578,58400,89.55,67,22807,45,1441,01-01-1984,Salaried,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
1,537409,47145,65550,73.23,67,22807,45,1502,31-07-1985,Self employed,...,0,0,1991,0,0,1,1yrs 11mon,1yrs 11mon,0,1


# EDA

## Nulls

In [89]:
# Some missing values for employment type
data.isna().sum() / len(data)

UNIQUEID                               0.000000
DISBURSED_AMOUNT                       0.000000
ASSET_COST                             0.000000
LTV                                    0.000000
BRANCH_ID                              0.000000
SUPPLIER_ID                            0.000000
MANUFACTURER_ID                        0.000000
CURRENT_PINCODE_ID                     0.000000
DATE_OF_BIRTH                          0.000000
EMPLOYMENT_TYPE                        0.032858
DISBURSAL_DATE                         0.000000
STATE_ID                               0.000000
EMPLOYEE_CODE_ID                       0.000000
MOBILENO_AVL_FLAG                      0.000000
AADHAR_FLAG                            0.000000
PAN_FLAG                               0.000000
VOTERID_FLAG                           0.000000
DRIVING_FLAG                           0.000000
PASSPORT_FLAG                          0.000000
PERFORM_CNS_SCORE                      0.000000
PERFORM_CNS_SCORE_DESCRIPTION          0

## Target

In [4]:
# Load Default is the target
# Little bit of class imbalance

data['LOAN_DEFAULT'].value_counts(normalize = True)

0    0.782929
1    0.217071
Name: LOAN_DEFAULT, dtype: float64

## Input

In [5]:
data['DATE_OF_BIRTH'] = pd.to_datetime(data['DATE_OF_BIRTH'], format = '%d-%m-%Y')

In [ ]:
credit_history_length = data['CREDIT_HISTORY_LENGTH'].str.split(' ', expand = True)
average_acct_length = data['AVERAGE_ACCT_AGE'].str.split(' ', expand = True)


months_credit_history_length = pd.Series(credit_history_length[1].str.extract('(\d+)')[0]).astype(int)
years_credit_history_length = pd.Series(credit_history_length[0].str.extract('(\d+)')[0]).astype(int) * 12

months_average_acct_length = pd.Series(average_acct_length[1].str.extract('(\d+)')[0]).astype(int)
years_average_acct_length = pd.Series(average_acct_length[0].str.extract('(\d+)')[0]).astype(int) * 12


data['CREDIT_HISTORY_LENGTH'] = months_credit_history_length + years_credit_history_length
data['AVERAGE_ACCT_AGE'] = months_average_acct_length + years_average_acct_length

In [72]:
data.dtypes

UNIQUEID                                        int64
DISBURSED_AMOUNT                                int64
ASSET_COST                                      int64
LTV                                           float64
BRANCH_ID                                       int64
SUPPLIER_ID                                     int64
MANUFACTURER_ID                                 int64
CURRENT_PINCODE_ID                              int64
DATE_OF_BIRTH                          datetime64[ns]
EMPLOYMENT_TYPE                                object
DISBURSAL_DATE                                 object
STATE_ID                                        int64
EMPLOYEE_CODE_ID                                int64
MOBILENO_AVL_FLAG                               int64
AADHAR_FLAG                                     int64
PAN_FLAG                                        int64
VOTERID_FLAG                                    int64
DRIVING_FLAG                                    int64
PASSPORT_FLAG               

## ID Columns

In [74]:
id_cols = [m for m in data.columns if 'ID' in m]
id_cols

['UNIQUEID',
 'BRANCH_ID',
 'SUPPLIER_ID',
 'MANUFACTURER_ID',
 'CURRENT_PINCODE_ID',
 'STATE_ID',
 'EMPLOYEE_CODE_ID',
 'VOTERID_FLAG']

In [76]:
# Will need to collapse some categories into "other"
for col in id_cols:
    print(data[col].value_counts())

420825    1
573390    1
443579    1
634411    1
497340    1
         ..
485740    1
591926    1
632244    1
647110    1
630213    1
Name: UNIQUEID, Length: 233154, dtype: int64
2      13138
67     11328
3       9230
5       9218
36      8832
       ...  
217      183
261      176
84       156
111       89
158       69
Name: BRANCH_ID, Length: 82, dtype: int64
18317    1432
15694    1300
15663    1275
17980    1268
14234    1258
         ... 
18631       1
23768       1
23038       1
20315       1
18107       1
Name: SUPPLIER_ID, Length: 2953, dtype: int64
86     109534
45      56626
51      27204
48      16710
49      10220
120      9658
67       2405
145       778
153        12
152         6
156         1
Name: MANUFACTURER_ID, dtype: int64
2578    1880
1446    1731
1515    1087
2989     934
2943     899
        ... 
5913       1
4010       1
2189       1
2141       1
4614       1
Name: CURRENT_PINCODE_ID, Length: 6698, dtype: int64
4     44870
3     34078
6     33505
13    17884
9   

## Numeric Columns

In [79]:
exclude = id_cols + ['LOAN_DEFAULT']

In [84]:
numeric_data = data.select_dtypes(include='number')
numeric_data = numeric_data.drop(columns = exclude)
numeric_data.head(2)

,DISBURSED_AMOUNT,ASSET_COST,LTV,MOBILENO_AVL_FLAG,AADHAR_FLAG,PAN_FLAG,DRIVING_FLAG,PASSPORT_FLAG,PERFORM_CNS_SCORE,PRI_NO_OF_ACCTS,...,SEC_CURRENT_BALANCE,SEC_SANCTIONED_AMOUNT,SEC_DISBURSED_AMOUNT,PRIMARY_INSTAL_AMT,SEC_INSTAL_AMT,NEW_ACCTS_IN_LAST_SIX_MONTHS,DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS,AVERAGE_ACCT_AGE,CREDIT_HISTORY_LENGTH,NO_OF_INQUIRIES
0,50578,58400,89.55,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,47145,65550,73.23,1,1,0,0,0,598,1,...,0,0,0,1991,0,0,1,23,23,0


In [86]:
for col in numeric_data.columns:
    print(numeric_data[col].describe())

count    233154.000000
mean      54356.993528
std       12971.314171
min       13320.000000
25%       47145.000000
50%       53803.000000
75%       60413.000000
max      990572.000000
Name: DISBURSED_AMOUNT, dtype: float64
count    2.331540e+05
mean     7.586507e+04
std      1.894478e+04
min      3.700000e+04
25%      6.571700e+04
50%      7.094600e+04
75%      7.920175e+04
max      1.628992e+06
Name: ASSET_COST, dtype: float64
count    233154.000000
mean         74.746530
std          11.456636
min          10.030000
25%          68.880000
50%          76.800000
75%          83.670000
max          95.000000
Name: LTV, dtype: float64
count    233154.0
mean          1.0
std           0.0
min           1.0
25%           1.0
50%           1.0
75%           1.0
max           1.0
Name: MOBILENO_AVL_FLAG, dtype: float64
count    233154.00000
mean          0.84032
std           0.36631
min           0.00000
25%           1.00000
50%           1.00000
75%           1.00000
max           1.0000